神经网络初始化
===
神经网络的初始化我们将它分为两个部分，分别是神经网络权重的初始化，以及输入数据的预处理

# 1.权重初始化
## 1.1.权重初始化的意义
模型权重的初始化对于网络的训练很重要, 不好的初始化参数会导致梯度传播问题, 降低训练速度; 而好的初始化参数, 能够加速收敛, 并且更可能找到较优解. 如果权重一开始很小，信号到达最后也会很小；如果权重一开始很大，信号到达最后也会很大。**不合适的权重初始化会使得隐藏层的输入的方差过大,从而在经过sigmoid这种非线性层时离中心较远(导数接近0),因此过早地出现梯度消失**.如使用均值0,标准差为1的正态分布初始化在隐藏层的方差仍会很大. 不初始化为0的原因是若初始化为0,所有的神经元节点开始做的都是同样的计算,最终同层的每个神经元得到相同的参数.好的初始化方法通常只是为了增快学习的速度(加速收敛),在某些网络结构中甚至能够提高准确率.

## 1.2.权重对于神经网络的影响

In [1]:
import torch

x = torch.randn(512)
for i in range(100):
    a = torch.randn(512, 512)
    x = a @ x
    if torch.isnan(x.std()):
        break;
i

28

可以看到，网络在第29层的时候发生了梯度爆炸，很明显网络的权重初始化值过大

In [2]:
import torch

x = torch.randn(512)
for i in range(100):
    a = torch.randn(512, 512) * 0.01
    x = a @ x
x.mean(), x.std()

(tensor(0.), tensor(0.))

可以看到，梯度发生了消失

## 1.3.高斯分布初始化$(0,0.01)$
初始化为小的随机数，如均值为0，方差为0.01的高斯分布，即用

In [2]:
import numpy as np
W = 0.01 * np.random.randn(2, 4)

这种初始化方法只适用于小型网络，对于深层次的网络，权重小道指反向传播计算中梯度也小，梯度"信号"被削弱。方差随着输入数量的增大而增大,可以通过正则化方差来提高权重收敛速率,初始权重的方式为正态分布

In [ ]:
import math
n = 64 #输出神经元的个数
w = np.random.randn(n) / math.sqrt(n)

这会使得中间结果$z=\sum_iw_ix_i+b$的方差较小,神经元不会饱和,学习速度不会减慢.

## 1.4.高斯分布初始化-$\frac{2}{n}$
ReLU激活函数喜欢的初始化形式为$\frac{2.0}{n}$，高斯分布权重初始化为如下

In [ ]:
import math
n = 64 #输出神经元的个数
w = np.random.randn(n) / math.sqrt(2.0 / n)

另外用形式$\frac{2}{n_{in} + n_{out}}$也是推荐的。不过对于以上两种方法，仅仅考虑的每层输入的方差，而后两种方式考虑输入与输出的方差，保持每层的输入与输出方差相等

## 1.5.Xavier初始化
Xavier初始化可以帮助减少梯度弥散问题，使得信号在神经网络中可以传递得更深。是最为常用的神经网络权重初始化方法。算法根据输入和输出神经元的数量自动决定初始化的范围，算法如下：
> 定义参数所在的层的输入维度为n,输出维度为m,那么参数将从$[-\sqrt{\frac{6}{m+n}, \sqrt{\frac{6}{m+n}}}]$均匀分布采样


假设输入一层X，输出一层Y，那么有
$$Y=W_1X_1+W_2X_2+...+W_nX_n$$
按照独立变量相乘的方差公式，可以计算出：
$$Var(W_iX_i)=E[X_i]^2Var(W_i)+E[W_i]^2Var(X_i)+Var(W_i)Var(X_i)$$
我们期望输入X和权重W都是零均值，因此简化为
$$Var(X_iX_i)=Var(W_i)Var(X_i)$$
进一步假设所有的$X_i,W_i$都是独立同分布，则有：
$$Var(Y)=Var(W_1X_1+W_2X_2+...+W_nX_n)=nVar(W_i)Var(X_i)$$
即输出的方差与输入有关，为使输出的方差与输入相同，意味着使$nVar(W_i)=1$，因此$Var(W_i)=\frac{1}{n}=\frac{1}{n_{in}}$。如果对反向传播的梯度运用同样的步骤，可得$Var(W_i)=\frac{1}{n_{out}}$。

由于$n_{in},n_{out}$通常不相等，所以这两个方差无法同时满足，作为一种折中的方案，可使用介于$\frac{1}{n_{in}},\frac{1}{n_{out}}$之间的数来代替：简单的选择是
$$Var(W_i)=\frac{2}{n_{in}+n_{out}}$$
可以根据均匀分布的方差，反推出W的均匀分布：
$$Var=\frac{(b-a)^2}{12}$$
使其零均值，则$b=-a$，有$Var=\frac{(2b)^2}{12}=\frac{2}{n_{in}+n_{out}}$，可得$b=\frac{\sqrt{6}}{\sqrt{n_{in}+n_{out}}}$，因此Xavier初始化的就是按照下面的均匀分布
$$W \sim U[-\sqrt{\frac{6}{m+n}, \sqrt{\frac{6}{m+n}}}]$$

In [5]:
import torch
import math

def xavier(m,h):
    return torch.Tensor(m,h).uniform_(-1, 1) * math.sqrt(6. / (m+h))

x = torch.randn(512)
for i in range(100):
    a = xavier(512, 512)
    x = torch.tanh(a @ x)

x.mean(), x.std()

(tensor(0.0011), tensor(0.0778))

## 1.6.MSRA初始化
MSRA方法是对于Xavier的改进。Xavier初始化方式为使每层方差一致，从而不会发生前向传播爆炸和反向传播梯度消失等问题。对于ReLU激活函数，其使一半数据变成0，初始时这一半的梯度为0，而tanh和sigmoid等的输出初始时梯度接近于1.因此使用ReLU的网络的参数方差可能会波动。
于是使用$Var(W)=\frac{2}{n_{in}}$放大一倍方差来保持方差的平稳。于是MSRA就是使用均值为0，方差为$\frac{4}{n_{in} + n_{out}}$的高斯分布

## 1.7.kaiming初始化
当使用关于0对称且在$[-1,1]$内有输出的激活函数(sigmoid或tanh)时，使用Xavier初始化比较合适。但是如果使用ReLU等激活函数会有什么效果呢？
- 使用适合给定图层的权重矩阵创建张量，并使用从标准正态分布中随机选择的数字填充它
- 将每个随机算则的数字乘以$\frac{\sqrt{2}}{\sqrt{n}}$，其中n是从前一层输出到指定层的连接数fan-in
- 偏差张量初始化为0

In [7]:
import torch
import math

def xavier(m,h):
    return torch.Tensor(m,h).uniform_(-1, 1) * math.sqrt(6. / (m+h))

x = torch.randn(512)
for i in range(100):
    a = xavier(512, 512)
    x = a @ x
    x = x.clamp_min(0.)

x.mean(), x.std()

(tensor(7.0151e-16), tensor(1.0094e-15))

第100层激活输出几乎完全消失了

In [8]:
import torch
import math

def kaiming(m,h):
    return torch.randn(m, h) * math.sqrt(2./m)

x = torch.randn(512)
for i in range(100):
    a = kaiming(512, 512)
    x = a @ x
    x = x.clamp_min(0.)

x.mean(), x.std()

(tensor(0.2182), tensor(0.3193))

使用kaiming初始化，有更好的表现

# 2.偏置初始化
通常将偏置初始化为0.若初始化为0.01等值,可能并不能得到好的提升,反而可能下降

In [ ]:
import torch.nn
net = torch.nn.Sequential()
for m in net.modules():
    if isinstance(m, torch.nn.Conv2d):
        m.weight.data.normal_(0, 0.02)
    elif isinstance(m, torch.nn.ConvTranspose2d):
        m.weight.data.normal_(0, 0.02)
    elif isinstance(m, torch.nn.BatchNorm2d):
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# 3.输入数据预处理
数据过来，需要在这一层进行一些数据预处理的操作，常见的3种数据预处理的方法有

## 3.1.去均值(Mean subtraction)
把输入数据各个维度都中心化到0(只是针对训练集！！！)，首先计算训练集的均值，然后把每一个图片都减去这个均值(测试集上也是减这个均值，不要去计算测试集上的均值)，主要原因是光照会影响.去均值也叫做均值减法，是晕处理最常用的形式。它对数据中每个独立特征减去平均值，从几何上可以理解为在每个维度上都将数据云的中心迁移到原点。在numpy中，该操作可以通过代码$X -= np.mean(X, axis=0)$实现。而对于图像，更常用的是对所有像素都减去一个值，可以用$X -= np.mean(X)$实现，也可以在3个颜色通道上分别操作

## 3.2.归一化Normalization
幅度归一化到一定的范围(CNN操作一般不做这个操作，因为RGB就是在0~255这个范围之内的)
1. 方法一：先对数据做零中心化(zero-centered)处理，然后每个维度都除以其标准差，$X /= np.std(X, axis=0)$
2. 方法二：对每个维度都做归一化，使得每个维度的最大和最小是1和-1

## 3.3.PCA/白化 Whitening
用PCA降维在这种处理中，先对数据进行零中心化处理，然后计算协方差矩阵，它展示了数据中的相关性结构$U,S,V=np.linalg.svd(cov)$。白化是对数据每个特征轴上的幅度归一化(CNN一般也不用)。白化操作的输入是特征基准上的数据，然后对每个维度除以其特征值来对数据范围进行归一化，几何解释是，如果数据服从多变量的高斯分布，经过白化后，数据的分布将会是一个均值为0，且协方差相等的矩阵。$Xwhite = \frac{Xrot}{np.sqrt(S + 1e-5)}$。<br/>
左边是原始数据，中间是做了PCA之后的数据，相当于先零中心化，然后做了旋转(去相关性)，右边是做了白化之后的操作

# 4.总结
- 当前的主流初始化方式Xavier,MSRA主要是为了保持每层的输入与输出方差相等, 而参数的分布采用均匀分布或高斯分布均可.
- 在广泛采用Batch Normalization的情况下, 使用普通的小方差的高斯分布即可.
- 另外, 在迁移学习的情况下, 优先采用预训练的模型进行参数初始化.

![images](Images/05_01_001.png)


# 5.使用Kaiming初始化方法初始化神经网络权重

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import time

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## 5.1.导入数据

In [4]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 1
learning_rate = 0.05
num_epochs = 10
batch_size = 128

# Architecture
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.MNIST(root='/input/', 
                               train=True, 
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='/input', 
                              train=False, 
                              transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


## 5.2.模型

In [5]:
class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        # calculate same padding:
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2
        
        # 28x28x1 => 28x28x4
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=4,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) # (1(28-1) - 28 + 3) / 2 = 1
        # 28x28x4 => 14x14x4
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(14-1) - 28 + 2) = 0                                       
        # 14x14x4 => 14x14x8
        self.conv_2 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) # (1(14-1) - 14 + 3) / 2 = 1                 
        # 14x14x8 => 7x7x8                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(7-1) - 14 + 2) = 0
        
        self.linear_1 = torch.nn.Linear(7*7*8, num_classes)
        
        ###############################################
        # Reinitialize weights using He initialization
        ###############################################
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.detach())
                m.bias.detach().zero_()
            elif isinstance(m, torch.nn.Linear):
                nn.init.kaiming_normal_(m.weight.detach())
                m.bias.detach().zero_()
        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)

        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)
        
        logits = self.linear_1(out.view(-1, 7*7*8))
        probas = F.softmax(logits, dim=1)
        return logits, probas

    
torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)

model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

## 5.3.训练

In [6]:
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = features.to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)

        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))
    
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))
    
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/469 | Cost: 2.4577
Epoch: 001/010 | Batch 050/469 | Cost: 1.1068
Epoch: 001/010 | Batch 100/469 | Cost: 0.6609
Epoch: 001/010 | Batch 150/469 | Cost: 0.5353
Epoch: 001/010 | Batch 200/469 | Cost: 0.4480
Epoch: 001/010 | Batch 250/469 | Cost: 0.3159
Epoch: 001/010 | Batch 300/469 | Cost: 0.4545
Epoch: 001/010 | Batch 350/469 | Cost: 0.4276
Epoch: 001/010 | Batch 400/469 | Cost: 0.1386
Epoch: 001/010 | Batch 450/469 | Cost: 0.1409
Epoch: 001/010 training accuracy: 91.97%
Time elapsed: 0.44 min
Epoch: 002/010 | Batch 000/469 | Cost: 0.2196
Epoch: 002/010 | Batch 050/469 | Cost: 0.1464
Epoch: 002/010 | Batch 100/469 | Cost: 0.2625
Epoch: 002/010 | Batch 150/469 | Cost: 0.1918
Epoch: 002/010 | Batch 200/469 | Cost: 0.1485
Epoch: 002/010 | Batch 250/469 | Cost: 0.1230
Epoch: 002/010 | Batch 300/469 | Cost: 0.1591
Epoch: 002/010 | Batch 350/469 | Cost: 0.1409
Epoch: 002/010 | Batch 400/469 | Cost: 0.1404
Epoch: 002/010 | Batch 450/469 | Cost: 0.1211
Epoch: 002/010 t

## 5.4.评估

In [7]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 97.67%
